In [1]:
import guidance
import datasets

/home/marcotcr/.virtualenvs/guidance/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
d = datasets.load_dataset('bigbench', 'anachronisms')

2023-04-04 11:08:54.175448: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-04 11:08:54.276968: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:/usr/local/opencv-3.1.0/lib:/usr/local/cuda-10.0/lib64:/usr/lib/x86_64-linux-gnu/openmpi/lib/
2023-04-04 11:08:54.276985: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-04-04 11:08:54.748229: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Coul

In [3]:
inputs = [x.split('\n')[0] for x in d['validation']['inputs']]
labels = [x[0] for x in d['validation']['targets']]

In [4]:
guidance.llm = guidance.llms.OpenAI("text-davinci-003") 

In [5]:
instruction = 'Given a sentence tell me whether it contains an anachronism (i.e. whether it could have happened or not based on the time periods associated with the entities).'
examples = [
    {'input': 'I wrote about shakespeare',
    'entities': [{'entity': 'I', 'time': 'present'}, {'entity': 'Shakespeare', 'time': '16th century'}],
    'reasoning': 'I can write about Shakespeare because he lived in the past with respect to me.',
    'answer': 'No'},
    {'input': 'Shakespeare wrote about me',
    'entities': [{'entity': 'Shakespeare', 'time': '16th century'}, {'entity': 'I', 'time': 'present'}],
    'reasoning': 'Shakespeare cannot have written about me, because he died before I was born',
    'answer': 'Yes'}
    ]

In [6]:
fewshot_prompt = guidance("""{{instruction}}
---{{#each examples}}
Sentence: {{this.input}}
Anachronism: {{this.answer}}
{{/each}}
Sentence: {{input}}
Anachronism:{{#select "answer"}} Yes{{or}} No{{/select}}""", llm=llm)
fewshot_prompt(examples=examples, instruction=instruction, input='I am a man')

NameError: name 'llm' is not defined

In [13]:
structure_prompt = guidance(
'''{{instruction}}
----
{{~! Few shot examples here ~}}
{{~#each examples}}
Sentence: {{this.input}}
Entities and dates:{{#each this.entities}}
{{this.entity}}: {{this.time}}{{/each}}
Reasoning: {{this.reasoning}}
Anachronism: {{this.answer}}
---
{{~/each}}
{{~! Input example here}}
Sentence: {{input}}
Entities and dates:
{{gen "entities"}}
Reasoning:{{gen "reasoning"}}
Anachronism:{{#select "answer"}} Yes{{or}} No{{/select}}''')
structure_prompt(examples=examples, input='The T-rex bit my dog', instruction=instruction)

Stop program Given a sentence tell me whether it contains an anachronism (i.e. whether it could have happened or not based on the time periods associated with the entities). 
----<span style='opacity: 1.0; display: inline; background-color: rgba(0, 138.56128016, 250.76166089, 0.25);' title='&#123;&#123;~#each examples&#125;&#125;
Sentence: &#123;&#123;this.input&#125;&#125;
Entities and dates:&#123;&#123;#each this.entities&#125;&#125;
&#123;&#123;this.entity&#125;&#125;: &#123;&#123;this.time&#125;&#125;&#123;&#123;/each&#125;&#125;
Reasoning: &#123;&#123;this.reasoning&#125;&#125;
Anachronism: &#123;&#123;this.answer&#125;&#125;
---
&#123;&#123;~/each&#125;&#125;'>
Sentence: I wrote about shakespeare 
Entities and dates: 
 I : present 
 Shakespeare : 16th century 
Reasoning: I can write about Shakespeare because he lived in the past with respect to me. 
Anachronism: No 
---
Sentence: Shakespeare wrote about me 
Entities and dates: 
 Shakespeare : 16th century 
 I : present 
Reasoning: Shakespeare cannot have written about me, because he died before I was born 
Anachronism: Yes 
--- 
Sentence: The T-rex bit my dog 
Entities and dates:
 T-rex: 65 million years ago
My dog: present 
Reasoning: The T-rex lived millions of years before my dog, so it cannot have bitten my dog. 
Anachronism: Yes

In [9]:
import numpy as np
input = inputs[0]
label = labels[0]
fews = []
structs = []
for input, label in zip(inputs, labels):
    f = fewshot_prompt(examples=examples, instruction=instruction, input=input)
    f = 'Yes' if 'Yes' in f['answer'] else 'No'
    s = structure_prompt(examples=examples, input=input, instruction=instruction)
    s = 'Yes' if 'Yes' in s['answer'] else 'No'
    fews.append(f)
    structs.append(s)
fews = np.array(fews)
structs = np.array(structs)

In [10]:
print('Few-shot', (np.array(labels) == fews).mean())
print('Structured output', (np.array(labels) == structs).mean())

Few-shot 0.6304347826086957
Structured output 0.7608695652173914
